In [1]:
# Setup the environment
!pip install -q -U immutabledict sentencepiece 
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
The syntax of the command is incorrect.
mv: cannot stat '/kaggle/working/gemma_pytorch/gemma/*': No such file or directory


In [13]:
!ls gemma_pytorch

CONTRIBUTING.md
LICENSE
README.md
docker
gemma
requirements.txt
scripts
setup.py
tokenizer


In [15]:
import sys 
sys.path.append("gemma_pytorch") 
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch

In [16]:
#ensure that this notebook is cuda-aware
torch.cuda.is_available()

True

In [17]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [18]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080 Ti'

In [28]:
# Load the model
VARIANT = "2b-it" 
# Need to set this to cuda, not gpu or cpu while using the gpu t4 on kaggle.
# Much faster results (as expected) when I did so.
MACHINE_TYPE = "cuda" 
weights_dir = 'gemma_pytorch\\tokenizer' 

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")


In [29]:
print(model_config)

GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='gemma_pytorch\\tokenizer\\tokenizer.model')


In [31]:

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()

D:\Dev\anaconda\envs\cuda_test\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [32]:
# Use the model

USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"
MODEL_CHAT_TEMPLATE = "<start_of_turn>model\n{prompt}<end_of_turn>\n"

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="Produce a check list of proper dead lift form and concerns"
    )
    + "<start_of_turn>model\n"
)

model.generate(
    prompt,
    device=device,
    output_len=300,
)

'**Proper Dead Lift Form**\n\n* Keep your back straight and your body in a neutral position.\n* Use a barbell that is appropriate for your body weight.\n* Press the barbell up until your arms are completely locked out.\n* Squeeze your glutes and hamstrings to return the barbell to the ground.\n\n**Concerns with Dead Lift Form**\n\n* **Poor posture:** Slouching or hunching over causes the spine to round, increasing the risk of injury.\n* **Not using enough weight:** Using too light of a weight makes it difficult to perform the exercise correctly.\n* **Not keeping your back straight:** A stiff back makes it easier for the barbell to come down too quickly, resulting in injury.\n* **Using too much weight:** Lifting too heavy can damage your muscles and joints.\n* **Not gripping the barbell properly:** Using a grip that is too wide or too narrow can cause the barbell to slide out of your hands and drop.\n* **Allowing the barbell to swing back too far:** This can cause overtraining and muscl